IMPORTS 

In [5]:
import numpy as np 
import pandas as pd 
import os 
from scipy.spatial.transform import Rotation as R 
import cv2 
import matplotlib.pyplot as plt
import json


FUNCTIONS

In [6]:
def pose_json_to_dict(pose_file): 
    with open(pose_file, 'r') as file:
        pose_dict_raw = json.load(file) 
    # go through each key and convert nested list to np array 
    pose_dict = {}
    for key in pose_dict_raw: 
        pose_dict[key] = np.array(pose_dict_raw[key])

    # compute output data 
    # compute incident angle of light between plane z axis and light -z axis 
    tag_plane_normal = pose_dict["tag"][:3,2] 
    light_direction = pose_dict["light"][:3,2]  
    cam_direction = -pose_dict["cam"][:3,2] 
    pose_dict["tag_light_angle_of_incidence"] = np.arccos(np.dot(tag_plane_normal, light_direction) / (np.linalg.norm(tag_plane_normal) * np.linalg.norm(light_direction))) * 180 / np.pi
    return pose_dict 

def metadata_json_to_dict(metadata_file): 
    with open(metadata_file, 'r') as file:
        metadata_dict = json.load(file) 
    return metadata_dict 

def lambertian_reflection(I_incident, N, L):
    # Calculate diffuse reflection (Lambertian model)
    return I_incident * max(np.dot(N, L), 0)

def phong_reflection(I_incident, N, L, V, shininess):
    # Calculate the reflection vector
    R = (2 * np.dot(N, L) * N) - L
    # Calculate specular reflection using the Phong model
    return I_incident * max(np.dot(R, V), 0) ** shininess

def process_data(metadata_dict, pose_dict): 
    tag_light_aoi = pose_dict["tag_light_angle_of_incidence"]  
    light_exposure = metadata_dict["light"]["exposure"] 
    light_color = metadata_dict["light"]["color"]   
    # diffuse_reflection = np.max((np.cos(tag_light_aoi * np.pi / 180),0)) * (2**light_exposure) 
    N = pose_dict["tag"][:3,2] 
    L = pose_dict["light"][:3,2] 
    V = pose_dict["cam"][:3,2] 
    I_incident = 2**light_exposure 
    shininess = 1.0 # NOTE: placeholder value 
    diffuse_reflection = lambertian_reflection(I_incident, N, L)     
    specular_reflection = phong_reflection(I_incident, N, L, V, shininess)
    metadata = {
        "angle of incidence": tag_light_aoi, 
        "light_exposure": light_exposure, 
        # "light_color": light_color, # FIXME: parse data 
        "diffuse_reflection": diffuse_reflection, 
        "specular_reflection": specular_reflection, 
    }
    return metadata 

READ DATA 

In [7]:
sdg_dir = "/media/rp/Elements/abhay_ws/marker_detection_failure_recovery/data/marker_obj_sdg/" 
# list all files ordered by time of creation 
files = os.listdir(sdg_dir) 
files.sort(key=lambda x: os.path.getmtime(os.path.join(sdg_dir, x))) 
data_dir = files[-1] # get latest data 
pose_dir = os.path.join(sdg_dir, data_dir, "pose") 
rgb_dir = os.path.join(sdg_dir, data_dir, "rgb") 
seg_dir = os.path.join(sdg_dir, data_dir, "seg") 
metadata_dir = os.path.join(sdg_dir, data_dir, "metadata")   

# get all files in the directories 
pose_files = os.listdir(pose_dir) 
rgb_files = os.listdir(rgb_dir) 
seg_files = os.listdir(seg_dir) 
metadata_files = os.listdir(metadata_dir) 

# filter seg_files to only include .png files 
seg_files = [f for f in seg_files if f.endswith(".png")]

# sort the files by time of creation
pose_files.sort(key=lambda x: os.path.getmtime(os.path.join(pose_dir, x))) 
rgb_files.sort(key=lambda x: os.path.getmtime(os.path.join(rgb_dir, x)))
seg_files.sort(key=lambda x: os.path.getmtime(os.path.join(seg_dir, x))) 
metadata_files.sort(key=lambda x: os.path.getmtime(os.path.join(metadata_dir, x))) 

PROCESS RESULTS

In [ ]:
# Create the directory for saving images if it doesn't exist
output_dir = os.path.join(sdg_dir, data_dir, "summary_images") 
os.makedirs(output_dir, exist_ok=True) 

# Loop through all indices (images)
for idx in range(len(rgb_files)):
    rgb_file = os.path.join(rgb_dir, rgb_files[idx])
    seg_file = os.path.join(seg_dir, seg_files[idx])
    pose_file = os.path.join(pose_dir, pose_files[idx])
    metadata_file = os.path.join(metadata_dir, metadata_files[idx]) 

    # Read data from files
    rgb_image = cv2.imread(rgb_file)
    seg_image = cv2.imread(seg_file, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    pose_dict = pose_json_to_dict(pose_file) 
    metadata_dict = metadata_json_to_dict(metadata_file) 
    metadata = process_data(metadata_dict, pose_dict)

    # Check if images are loaded correctly
    if rgb_image is None:
        raise ValueError(f"RGB image at {rgb_file} could not be loaded.")
    if seg_image is None:
        raise ValueError(f"Segmentation image at {seg_file} could not be loaded.")

    # Convert from BGR (OpenCV default) to RGB (for matplotlib)
    image_rgb = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)

    # Create a new figure for each image
    plt.figure(figsize=(12, 6))  # Adjust figure size to make space for metadata

    # Subplot for RGB image
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
    plt.imshow(image_rgb)
    plt.axis('off')  # Hide axes
    plt.title(f'RGB Image {idx}')

    # Subplot for segmentation image
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
    plt.imshow(seg_image, cmap='viridis')  # Use a colormap for better visualization
    plt.axis('off')  # Hide axes
    plt.title(f'Segmentation Image {idx}')

    # Display metadata as text in a separate area (outside the main image area)
    metadata_str = '\n'.join([f'{key}: {value:.2f}' for key, value in metadata.items()])

    # Create a new subplot for metadata
    plt.subplot(1, 2, 2)
    plt.text(1.05, 0.5, metadata_str, fontsize=12, ha='left', va='center', transform=plt.gca().transAxes, 
             bbox=dict(facecolor='white', alpha=0.7, edgecolor='black', boxstyle='round,pad=1'))

    # Adjust layout to avoid overlap and make space for metadata
    plt.tight_layout()  # Adjust layout
    plt.subplots_adjust(right=0.8)  # Make space for metadata on the right

    # Save the image to the summary_images folder
    save_path = os.path.join(output_dir, f"summary_image_{idx}.png")
    plt.savefig(save_path, bbox_inches='tight', dpi=300)  # Save with high resolution
    plt.close()  # Close the plot to free up memory

    if len(rgb_files) < 10 or (idx + 1) % (len(rgb_files) // 10) == 0:
        print(f"Saved image {idx} to {save_path}") 


Saved image 0 to /media/rp/Elements/abhay_ws/marker_detection_failure_recovery/data/marker_obj_sdg/markers_20250222-220540/summary_images/summary_image_0.png
Saved image 1 to /media/rp/Elements/abhay_ws/marker_detection_failure_recovery/data/marker_obj_sdg/markers_20250222-220540/summary_images/summary_image_1.png
Saved image 2 to /media/rp/Elements/abhay_ws/marker_detection_failure_recovery/data/marker_obj_sdg/markers_20250222-220540/summary_images/summary_image_2.png
Saved image 3 to /media/rp/Elements/abhay_ws/marker_detection_failure_recovery/data/marker_obj_sdg/markers_20250222-220540/summary_images/summary_image_3.png
Saved image 4 to /media/rp/Elements/abhay_ws/marker_detection_failure_recovery/data/marker_obj_sdg/markers_20250222-220540/summary_images/summary_image_4.png
Saved image 5 to /media/rp/Elements/abhay_ws/marker_detection_failure_recovery/data/marker_obj_sdg/markers_20250222-220540/summary_images/summary_image_5.png
Saved image 6 to /media/rp/Elements/abhay_ws/marker_